In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough # 이 강의의 학습 목표.
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

"""
강의 요약

chain.invoke 형태로 프로그램을 변경하기 위한 작업. LCEL 형태겠지?

0506 에서 이 소스로 변경을 하는 강의인데... 너무 어렵당.
1. chain 을 만들고, RunnablePassthrough 를 통해서 chat_history 를 매번 전달하도록 세팅한다.
2. load_memory 함수를 통해서 chat_history 를 가져와서 RunnablePassthrough 에 전달. 해당 정보를 바로 prompt 에 전달하게 된다.
3. invoke_chain 함수를 통해서, 호출 메시지를 간결하게 처리한다.
    chain.invoke 를 통해서. 질문을 던지고, 응답을 받는다. 질문던질때 chaining 을 통해서 위의 설정값을 세팅해서 전달.
    질문과 응답을 메모리에 설정해주는 작업을 해준다.(다음 질문을 위해서...)
전체 절차는 어려운건 아닌데...
"""

llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history", # 이 값이 아래 정보로 활용되도록 하는 방법이겠지? default:history
    return_messages=True,
)

# template = """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]

# chain = prompt | llm
chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [ ]:
invoke_chain("My name is nico.")

In [ ]:
invoke_chain(question="What is my name?")

In [ ]:
invoke_chain(question="Today's weather in Seoul is sunny.")

In [ ]:
# 왜 답을 재대로 하지 못할까???

invoke_chain(question="How's the weather where I live today?")